## Timetabling Program for SMUS Middle School
#### Copyright © 2020 Hoshino Math Services

In [ ]:
# import sys
# !{sys.executable} -m pip install ortools 

# import modules
import time
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [ ]:
# Import all the Data for Grades 6-8
InputFile = pd.ExcelFile("Middle School Input.xlsx") 
Grade6Matrix = pd.read_excel(InputFile, 'Grade6') 
Grade6Info = Grade6Matrix.values.tolist()
Grade7Matrix = pd.read_excel(InputFile, 'Grade7') 
Grade7Info = Grade7Matrix.values.tolist()
Grade8Matrix = pd.read_excel(InputFile, 'Grade8') 
Grade8Info = Grade8Matrix.values.tolist()


# Generate the set of Courses and Teachers at the Middle School
# NOTE: For now, we have assumed that in Grades 7 and 8, there will be 2 sections of Mandarin,
# 1 section of Computer Science, and 1 section of Communicaton Skills.  If this information
# is incorrect, we will change it on the Excel Input sheet.

AllCourses = []
AllTeachers = []
for j in [1,5,9,13]:
    for i in range(2, len(Grade6Info)-1):
        Course = Grade6Info[i][j]
        Teacher = Grade6Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
    for i in range(2, len(Grade7Info)-1):
        Course = Grade7Info[i][j]
        Teacher = Grade7Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
    for i in range(2, len(Grade8Info)-1):
        Course = Grade8Info[i][j]
        Teacher = Grade8Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
            
            
# For each (Grade, Section, Course) triplet, determine the Teacher of that class.

GSCTeacher = [ [['Not Applicable' for c in range(40)] for s in range(5)] for g in range(10)]
for j in [1,5,9,13]:
    s = int((j+3)/4)
    for i in range(2, len(Grade6Info)-1):
        Course = Grade6Info[i][j]
        Teacher = Grade6Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[6][s][c] = Teacher
    for i in range(2, len(Grade7Info)-1):
        Course = Grade7Info[i][j]
        Teacher = Grade7Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[7][s][c] = Teacher
    for i in range(2, len(Grade8Info)-1):
        Course = Grade8Info[i][j]
        Teacher = Grade8Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[8][s][c] = Teacher

In [ ]:
# Optimize assignment of courses to blocks


solver = pywraplp.Solver('SMUS Middle School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
start_time = time.time()

# Define our Grades, Sections, Days, Periods, Courses
Grades = [6,7,8]
Sections = [1,2,3,4]
Days = [1,2,3,4,5]
Periods = [1,2,3,4,5,6,7]
Courses = range(len(AllCourses))


# Define our five-dimensional Boolean Variables
x = {}
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                for c in Courses:
                    x[g,s,d,p,c] = solver.IntVar(0,1, 'x[%d,%d,%d,%d,%d]' % (g,s,d,p,c))

        
# CONSTRAINT 1: For each grade and each section, there is exactly one course each period.
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                solver.Add( sum(x[g,s,d,p,c] for c in Courses) == 1)
        
                        
# CONSTRAINT 2: Each course must be offered a set number of lessons in each week  
for s in Sections:
    for i in range(2, len(Grade6Info)-1):
        CourseName = Grade6Info[i][4*s-3]
        Lessons = Grade6Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[6,s,d,p,c] for d in Days for p in Periods) == Lessons)
    for i in range(2, len(Grade7Info)-1):
        CourseName = Grade7Info[i][4*s-3]
        Lessons = Grade7Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[7,s,d,p,c] for d in Days for p in Periods) == Lessons)
    for i in range(2, len(Grade8Info)-1):
        CourseName = Grade8Info[i][4*s-3]
        Lessons = Grade8Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[8,s,d,p,c] for d in Days for p in Periods) == Lessons)

        
# CONSTRAINT 3: Certain courses must be offered in certain blocks
# Chapel on Tuesday Period 1, Assembly on Friday Period 1, XPLO on Friday Period 7
for g in Grades:
    for s in Sections:
        c = AllCourses.index('Chapel')
        solver.Add( x[g,s,2,1,c] == 1)
        c = AllCourses.index('Assembly')
        solver.Add( x[g,s,5,1,c] == 1)
        c = AllCourses.index('XPLO')
        solver.Add( x[g,s,5,7,c] == 1)


# CONSTRAINT 4: No teacher can teach two different classes simultaneously
# CONSTRAINT 5: No teacher teaches more than FOUR lessons on any given day
for Teacher in AllTeachers:
    Tset = []
    for g in Grades:
        for s in Sections:
            for c in Courses:    
                if GSCTeacher[g][s][c] == Teacher:
                    GradeSectionCourse = [g, s, c]
                    Tset.append(GradeSectionCourse)
    
    for d in Days:
        for p in Periods:
            solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]] 
                           for z in range(len(Tset))) <= 1)

    for d in Days:
        solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]]
                       for z in range(len(Tset)) for p in Periods) <= 4)
       
                  
# CONSTRAINT 6: A student can't have two lessons in the same class in non-consecutive periods.
# This constraint automatically implies that each class can be offered at most twice per day.
for p1 in Periods:
    for p2 in Periods:
        if p2-p1>1:
            for g in Grades:
                for s in Sections:
                    for d in Days:
                        for c in Courses:
                            solver.Add(x[g,s,d,p1,c] + x[g,s,d,p2,c] <=1)

                            
# CONSTRAINT 7: Other than Hum, Math/Science, and Science, no multi-lesson class can be 
# offered more than once per day to any cohort.
for CourseName in ['French', 'Mandarin', 'Band', 'Strings', 'PE', 'Comp/Comm', 'Science', 'Art']:
    c = AllCourses.index(CourseName)
    for g in Grades:
        for s in Sections:
            for d in Days:
                solver.Add( sum(x[g,s,d,p,c] for p in Periods) <= 1)


# CONSTRAINT 8A: All Grade 6/7/8 students must have at least one Humanities class each day.
c = AllCourses.index('Humanities')
for g in [6,7,8]:
    for s in Sections:
        for d in Days:
            solver.Add( sum(x[g,s,d,p,c] for p in Periods) >= 1)

            
# CONSTRAINT 8B: All Grade 6/7 students must have at least one Math/Sci class each day.
c = AllCourses.index('Math/Science')
for g in [6,7]:
    for s in Sections:
        for d in Days:
            solver.Add( sum(x[g,s,d,p,c] for p in Periods) >= 1)
                    
                            
# CONSTRAINT 9A: Band/Strings lessons must take place in the following periods:
# Monday 5-7, Tuesday 1-7, Wednesday 5-7, Thursday 1-7
for c in [AllCourses.index('Band'), AllCourses.index('Strings')]:
    for g in Grades:
        for s in Sections:
            solver.Add( sum(x[g,s,1,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,3,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,5,p,c] for p in [1,2,3,4,5,6,7]) == 0)

for c in [AllCourses.index('SharedBand'), AllCourses.index('SharedStrings')]:
    for g in Grades:
        for s in Sections:
            solver.Add( sum(x[g,s,1,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,3,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,5,p,c] for p in [1,2,3,4,5,6,7]) == 0)
    

# CONSTRAINT 9B: For each grade, Band/Strings take place in the same periods for cohorts A+C, and cohorts B+D
c1 = AllCourses.index('Band')
c2 = AllCourses.index('Strings')
for g in Grades:
    for d in Days:
        for p in Periods:
            solver.Add( x[g,1,d,p,c1] == x[g,3,d,p,c2])
            solver.Add( x[g,2,d,p,c1] == x[g,4,d,p,c2])

            
# CONSTRAINT 9C: For each grade, SharedBand (cohorts A+B) and SharedStrings (cohorts C+D) take 
# place at the exact same period once per week.
c3 = AllCourses.index('SharedBand')
c4 = AllCourses.index('SharedStrings')
for g in Grades:
    for d in Days:
        for p in Periods:
            solver.Add( x[g,1,d,p,c3] == x[g,2,d,p,c3])
            solver.Add( x[g,2,d,p,c3] == x[g,3,d,p,c4])
            solver.Add( x[g,3,d,p,c4] == x[g,4,d,p,c4])
    

# CONSTRAINT 9D: Ensure Enns and Smith are not teaching two lessons (e.g. Band + SharedBand)
# in the same period
c1 = AllCourses.index('Band')
c2 = AllCourses.index('Strings')
c3 = AllCourses.index('SharedBand')
c4 = AllCourses.index('SharedStrings')
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                solver.Add(x[g,s,d,p,c1] + x[6,1,d,p,c3] <= 1)
                solver.Add(x[g,s,d,p,c1] + x[7,1,d,p,c3] <= 1)
                solver.Add(x[g,s,d,p,c1] + x[8,1,d,p,c3] <= 1)
                solver.Add(x[g,s,d,p,c2] + x[6,3,d,p,c4] <= 1)
                solver.Add(x[g,s,d,p,c2] + x[7,3,d,p,c4] <= 1)
                solver.Add(x[g,s,d,p,c2] + x[8,3,d,p,c4] <= 1)
                    
                    
# CONSTRAINT 10: Choir lessons must take place in the following periods:
# Monday 1-2, Tuesday 1-7, Thursday 5-7, Friday 1-7.  Also no Grade 8 lessons during Period 7.
c = AllCourses.index('Choir')
for g in Grades:
        for s in Sections:
            solver.Add( sum(x[g,s,1,p,c] for p in [3,4,5,6,7]) == 0)
            solver.Add( sum(x[g,s,3,p,c] for p in [1,2,3,4,5,6,7]) == 0)
            solver.Add( sum(x[g,s,4,p,c] for p in [1,2,3,4]) == 0)
for s in Sections:
    for d in Days:
        solver.Add( x[8,s,d,7,c] == 0)


# CONSTRAINT 11: For each grade, French must take place in the same periods for cohorts A+C, and cohorts B+D
# Furthermore, the lesson for cohort B+D is immediately after the lesson for cohort A+C
c = AllCourses.index('French')
for g in Grades:
    for d in Days:
        for p in Periods:
            solver.Add( x[g,1,d,p,c] == x[g,3,d,p,c])
            solver.Add( x[g,2,d,p,c] == x[g,4,d,p,c])
        
        solver.Add(x[g,1,d,7,c] == 0)
        solver.Add(x[g,2,d,1,c] == 0)
        
        for p in [1,2,3,4,5,6]:
            solver.Add( x[g,1,d,p,c] == x[g,2,d,p+1,c])
        
        
# CONSTRAINT 12A: Wilkins cannot teach three Art classes on any afternoon.
c = AllCourses.index('Art')
for d in Days:
    solver.Add(sum(x[g,s,d,p,c] for g in Grades for s in Sections for p in [5,6,7]) <= 2)
        

# CONSTRAINT 12B: Grade 8 students cannot have Art lessons on two consecutive days.
c = AllCourses.index('Art')
for s in Sections:
    for d in [1,2,3,4]:
        solver.Add(sum(x[8,s,d,p,c] + x[8,s,d+1,p,c] for p in Periods) <= 1)
   
           
# CONSTRAINT 13A: Students cannot have French on three consecutive days.
c = AllCourses.index('French')
for g in Grades:
    for s in Sections:
        for d in [1,2,3]:
            solver.Add(sum(x[g,s,d,p,c] + x[g,s,d+1,p,c] + x[g,s,d+2,p,c] for p in Periods) <= 2)
            

# CONSTRAINT 14: Students cannot have Mandarin on two consecutive days.
c = AllCourses.index('Mandarin')
for g in Grades:
    for s in Sections:
        for d in [1,2,3,4]:
            solver.Add(sum(x[g,s,d,p,c] + x[g,s,d+1,p,c] for p in Periods) <= 1)
        

    
# SET PREFERENCE COEFFICIENTS
w = [[[[[1 for c in range(40)] for p in range(10)] for d in range(10)] for s in range(10)] for g in range(10)] 


# PREFERENCE 1: More HUM and MATH/SCI lessons in the morning, prioritizing Grades 6 and 7 over Grade 8.

for s in Sections:
    for d in Days:
        for p in [1,2,3,4]:
            for c in [AllCourses.index('Humanities'), AllCourses.index('Math/Science')]:
                w[6][s][d][p][c]=3
                w[7][s][d][p][c]=3
                w[8][s][d][p][c]=2
                

                
                
# RUN OPTIMIZATION

solver.Maximize(solver.Sum(w[g][s][d][p][c]*x[g,s,d,p,c] for g in Grades for s in Sections for d in Days
                           for p in Periods for c in Courses))
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The code ran in', round(solving_time,1), 'seconds')

In [ ]:
CohortNames = ["-", "A", "B", "C", "D"]
OurColumns = ["Period", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

Timetable = [ [ "-" for d in range(6)] for z in range(28)]
for z in range(28):
    Timetable[z][0] = "P" + str(1+int(z/4))
for d in Days:
    for p in Periods:
        for s in Sections:
            for c in Courses:
                if x[6,s,d,p,c].solution_value()==1:
                    CourseID = str(6) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = " (" + GSCTeacher[6][s][c] + ")"
                    RowIndex = 4*(p-1)+(s-1)                        
                    Timetable[RowIndex][d] = CourseID + TeacherID
Grade6 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for d in range(6)] for z in range(28)]
for z in range(28):
    Timetable[z][0] = "P" + str(1+int(z/4))
for d in Days:
    for p in Periods:
        for s in Sections:
            for c in Courses:
                if x[7,s,d,p,c].solution_value()==1:
                    CourseID = str(7) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = " (" + GSCTeacher[7][s][c] + ")"
                    RowIndex = 4*(p-1)+(s-1)                        
                    Timetable[RowIndex][d] = CourseID + TeacherID
Grade7 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for d in range(6)] for z in range(28)]
for z in range(28):
    Timetable[z][0] = "P" + str(1+int(z/4))
for d in Days:
    for p in Periods:
        for s in Sections:
            for c in Courses:
                if x[8,s,d,p,c].solution_value()==1:
                    CourseID = str(8) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = " (" + GSCTeacher[8][s][c] + ")"
                    RowIndex = 4*(p-1)+(s-1)                        
                    Timetable[RowIndex][d] = CourseID + TeacherID
Grade8 = pd.DataFrame(Timetable, columns=OurColumns)

with pd.ExcelWriter('Optimal Middle School Timetable by Grade.xlsx') as writer:  
    Grade6.to_excel(writer, sheet_name='Grade6', index=False)
    Grade7.to_excel(writer, sheet_name='Grade7', index=False)
    Grade8.to_excel(writer, sheet_name='Grade8', index=False)

In [ ]:
AllTeachers.sort()

CohortNames = ["-", "A", "B", "C", "D"]
OurColumns = ["Period"] + AllTeachers

Timetable = [ [ "-" for t in range(len(AllTeachers)+1)] for z in range(7)]
for z in range(7):
    Timetable[z][0] = "P" + str(1+z)
for g in Grades:
    for s in Sections:
        for p in Periods:
            for c in Courses:
                if x[g,s,1,p,c].solution_value()==1:
                    CourseID = str(g) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = GSCTeacher[g][s][c]
                    if TeacherID != 'Not Applicable':
                        t = AllTeachers.index(TeacherID)
                        Timetable[p-1][t+1] = CourseID
Day1 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for t in range(len(AllTeachers)+1)] for z in range(7)]
for z in range(7):
    Timetable[z][0] = "P" + str(1+z)
for g in Grades:
    for s in Sections:
        for p in Periods:
            for c in Courses:
                if x[g,s,2,p,c].solution_value()==1:
                    CourseID = str(g) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = GSCTeacher[g][s][c]
                    if TeacherID != 'Not Applicable':
                        t = AllTeachers.index(TeacherID)
                        Timetable[p-1][t+1] = CourseID
Day2 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for t in range(len(AllTeachers)+1)] for z in range(7)]
for z in range(7):
    Timetable[z][0] = "P" + str(1+z)
for g in Grades:
    for s in Sections:
        for p in Periods:
            for c in Courses:
                if x[g,s,3,p,c].solution_value()==1:
                    CourseID = str(g) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = GSCTeacher[g][s][c]
                    if TeacherID != 'Not Applicable':
                        t = AllTeachers.index(TeacherID)
                        Timetable[p-1][t+1] = CourseID
Day3 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for t in range(len(AllTeachers)+1)] for z in range(7)]
for z in range(7):
    Timetable[z][0] = "P" + str(1+z)
for g in Grades:
    for s in Sections:
        for p in Periods:
            for c in Courses:
                if x[g,s,4,p,c].solution_value()==1:
                    CourseID = str(g) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = GSCTeacher[g][s][c]
                    if TeacherID != 'Not Applicable':
                        t = AllTeachers.index(TeacherID)
                        Timetable[p-1][t+1] = CourseID
Day4 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for t in range(len(AllTeachers)+1)] for z in range(7)]
for z in range(7):
    Timetable[z][0] = "P" + str(1+z)
for g in Grades:
    for s in Sections:
        for p in Periods:
            for c in Courses:
                if x[g,s,5,p,c].solution_value()==1:
                    CourseID = str(g) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = GSCTeacher[g][s][c]
                    if TeacherID != 'Not Applicable':
                        t = AllTeachers.index(TeacherID)
                        Timetable[p-1][t+1] = CourseID
Day5 = pd.DataFrame(Timetable, columns=OurColumns)

with pd.ExcelWriter('Optimal Middle School Timetable by Teacher.xlsx') as writer:  
    Day1.to_excel(writer, sheet_name='Day1', index=False)
    Day2.to_excel(writer, sheet_name='Day2', index=False)
    Day3.to_excel(writer, sheet_name='Day3', index=False)
    Day4.to_excel(writer, sheet_name='Day4', index=False)
    Day5.to_excel(writer, sheet_name='Day5', index=False)